This notebook does the following on the Polypharmacy dataset:

1. Load drug and protein embeddings from NCMF and DCMF
2. Load single side effect dataset and polypharmacy side effect dataset
3. Make k clusters of the drug embeddings(single drug embedding or concatenated drug embedding) - PCA+KMeans
4. Choose top k categories of side effects from polypharmacy, top k side effect names from single side effect dataset.
5. Plot distribution of k side effects/ k categories in each case
6. Calculate silhouette and ARI scores
7. Visualize clusters using hypertools (TSNE, Spectral, Incremental PCA)
8. Visualize groups based on existing labels

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
k_single = 5
k_poly = 5
topk_single = 5
topk_poly =5

#### Step 1 Load embeddings for drugs and proteins - NCMF and DCMF

In [3]:
def load(emb_file_path):
    emb_dict = {}
    with open(emb_file_path, 'r') as emb_file:
        for i, line in enumerate(emb_file):
            if i == 0:
                train_para = line[:-1]
            else:
                index, emb = line[:-1].split('\t')
                emb_dict[index] = np.array(emb.split()).astype(np.float32)

    return train_para, emb_dict

In [4]:
ncmf_emb_file = "../datasets/NCMF/Polypharmacy/emb_sample_1.dat"
dcmf_emb_file = "../DCMF/experiments/emb_Polypharmacy_sample_1.dat"
dfmf_emb_file = "../scikit-fusion/experiments/dfmf_s1_emb_polypharmacy.dat"
cmf_emb_file = "../CMF/experiments/cmf_s1_polypharmacy_emb.dat"
gcmf_emb_file = "../CMF/experiments/gcmf_s1_emb_polypharmacy.dat"

In [5]:
train_para, emb_dict = load(ncmf_emb_file)

In [6]:
train_para_dcmf, emb_dict_dcmf = load(dcmf_emb_file)

In [7]:
train_para_dfmf, emb_dict_dfmf = load(dfmf_emb_file)
train_para_cmf, emb_dict_cmf = load(cmf_emb_file)
train_para_gcmf, emb_dict_gcmf = load(gcmf_emb_file)

In [8]:
print(emb_dict_dfmf['0'].shape)
print(emb_dict_cmf['0'].shape)
print(emb_dict_gcmf['0'].shape)

(50,)
(50,)
(50,)


#### Step 2 - Load single side effect and polypharmacy datasets

In [9]:
all_drugs_df = pd.read_csv("./drug2ID.csv", index_col=0)
all_drugs_df.head()

,Drug ID
Drug Name,
CID000000085,0
CID000000119,1
CID000000143,2
CID000000158,3
CID000000159,4


In [10]:
def drug2IDmap(x):
    drug_id_for_x = all_drugs_df.loc[x]["Drug ID"]
    return drug_id_for_x

In [11]:
single_se_df = pd.read_csv("./bio-decagon-mono.csv")
single_se_df.head()

,STITCH,Individual Side Effect,Side Effect Name
0,CID003062316,C1096328,central nervous system mass
1,CID003062316,C0162830,Photosensitivity reaction
2,CID003062316,C1611725,leukaemic infiltration brain
3,CID003062316,C0541767,platelet adhesiveness abnormal
4,CID003062316,C0242973,Ventricular dysfunction


In [12]:
single_se_df["drug"] = single_se_df["STITCH"].apply(lambda x: drug2IDmap(x))
single_se_df.head()

,STITCH,Individual Side Effect,Side Effect Name,drug
0,CID003062316,C1096328,central nervous system mass,608
1,CID003062316,C0162830,Photosensitivity reaction,608
2,CID003062316,C1611725,leukaemic infiltration brain,608
3,CID003062316,C0541767,platelet adhesiveness abnormal,608
4,CID003062316,C0242973,Ventricular dysfunction,608


In [13]:
ddi_df = pd.read_csv("./bio-decagon-combo.csv")
ddi_df.head()

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache


In [14]:
ddi_df["drugA"] = ddi_df["STITCH 1"].apply(lambda x:drug2IDmap(x))
ddi_df.head()

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name,drugA
0,CID000002173,CID000003345,C0151714,hypermagnesemia,55
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity,55
2,CID000002173,CID000003345,C0004144,atelectasis,55
3,CID000002173,CID000003345,C0002063,alkalosis,55
4,CID000002173,CID000003345,C0004604,Back Ache,55


In [ ]:
ddi_df["drugB"] = ddi_df["STITCH 2"].apply(lambda x:drug2IDmap(x))
ddi_df.head()

In [ ]:
### Reading the side effect category file from SNAP
se_file = pd.read_csv("./bio-decagon-effectcategories.csv")
se_file.head()

In [ ]:
def get_se_category(x):
    res = list(se_file[se_file["Side Effect"] == x]["Disease Class"])
    if res == []:
        return "No information"
    else:
        return res[0]

In [ ]:
ddi_df["side_effect_category"] = ddi_df["Polypharmacy Side Effect"].apply(lambda x: get_se_category(x))
ddi_df.head()

In [ ]:
ddi_df = ddi_df[ddi_df.side_effect_category != "No information"]
ddi_df.head()

In [ ]:
single_se_df_subset = single_se_df.drop_duplicates("STITCH")
print(single_se_df.shape)
print(single_se_df_subset.shape)
single_se_df_subset.head()

In [ ]:
len(single_se_df_subset["Side Effect Name"].unique())

In [ ]:
ddi_df_subset = ddi_df.drop_duplicates(["drugA", "drugB"])
print(ddi_df.shape)
print(ddi_df_subset.shape)
ddi_df_subset.head()

In [ ]:
len(ddi_df_subset["side_effect_category"].unique())

#### Step 3 - get drug embeddings and obtain k clusters using PCA + KMeans/hypertools

In [ ]:
def get_emb(drug_ID, alg):
    if alg == "ncmf":
        left = emb_dict[str(drug_ID)]
    elif alg == "dcmf":
        left = emb_dict_dcmf[str(drug_ID)]
    elif alg == "dfmf":
        left = emb_dict_dfmf[str(drug_ID)]
    elif alg == "cmf":
        left = emb_dict_cmf[str(drug_ID)]
    elif alg == "gcmf":
        left = emb_dict_gcmf[str(drug_ID)]
    return list(left)

##### Single Side effect

In [ ]:
single_se_df_subset["drug_emb_ncmf"] = single_se_df_subset["drug"].apply(lambda x: get_emb(x, "ncmf"))
single_se_df_subset.head()

In [ ]:
single_se_df_subset["drug_emb_dcmf"] = single_se_df_subset["drug"].apply(lambda x: get_emb(x, "dcmf"))
single_se_df_subset.head()

In [ ]:
single_se_df_subset["drug_emb_cmf"] = single_se_df_subset["drug"].apply(lambda x: get_emb(x, "cmf"))
single_se_df_subset["drug_emb_gcmf"] = single_se_df_subset["drug"].apply(lambda x: get_emb(x, "gcmf"))
single_se_df_subset["drug_emb_dfmf"] = single_se_df_subset["drug"].apply(lambda x: get_emb(x, "dfmf"))

In [ ]:
single_se_df_subset[[f"emb_ncmf_{i}" for i in range(50)]] = pd.DataFrame(single_se_df_subset.drug_emb_ncmf.tolist(), index=single_se_df_subset.index)
single_se_df_subset[[f"emb_dcmf_{i}" for i in range(100)]] = pd.DataFrame(single_se_df_subset.drug_emb_dcmf.tolist(), index=single_se_df_subset.index)
single_se_df_subset[[f"emb_cmf_{i}" for i in range(50)]] = pd.DataFrame(single_se_df_subset.drug_emb_cmf.tolist(), index=single_se_df_subset.index)
single_se_df_subset[[f"emb_gcmf_{i}" for i in range(50)]] = pd.DataFrame(single_se_df_subset.drug_emb_gcmf.tolist(), index=single_se_df_subset.index)
single_se_df_subset[[f"emb_dfmf_{i}" for i in range(50)]] = pd.DataFrame(single_se_df_subset.drug_emb_dfmf.tolist(), index=single_se_df_subset.index)
single_se_df_subset.head()

In [ ]:
import hypertools as hyp

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5, random_state=42)
single_se_ncmf_pca = pca.fit_transform(single_se_df_subset[[f"emb_ncmf_{i}" for i in range(50)]].values)
single_se_dcmf_pca = pca.fit_transform(single_se_df_subset[[f"emb_dcmf_{i}" for i in range(100)]].values)
single_se_cmf_pca = pca.fit_transform(single_se_df_subset[[f"emb_cmf_{i}" for i in range(50)]].values)
single_se_gcmf_pca = pca.fit_transform(single_se_df_subset[[f"emb_gcmf_{i}" for i in range(50)]].values)
single_se_dfmf_pca = pca.fit_transform(single_se_df_subset[[f"emb_dfmf_{i}" for i in range(50)]].values)
print(single_se_ncmf_pca.shape)
print(single_se_dcmf_pca.shape)
print(single_se_cmf_pca.shape)
print(single_se_gcmf_pca.shape)
print(single_se_dfmf_pca.shape)

In [ ]:
single_se_df_subset["ncmf_clusters"] = hyp.cluster(single_se_ncmf_pca, n_clusters = k_single)

In [ ]:
single_se_df_subset["dcmf_clusters"] = hyp.cluster(single_se_dcmf_pca, n_clusters = k_single)

In [ ]:
single_se_df_subset["cmf_clusters"] = hyp.cluster(single_se_cmf_pca, n_clusters = k_single)
single_se_df_subset["gcmf_clusters"] = hyp.cluster(single_se_gcmf_pca, n_clusters = k_single)
single_se_df_subset["dfmf_clusters"] = hyp.cluster(single_se_dfmf_pca, n_clusters = k_single)

In [ ]:
single_se_df_subset.head()

##### Polypharmacy side effect

In [ ]:
ddi_df_subset["drug_emb_ncmf"] = ddi_df_subset["drugA"].apply(lambda x: get_emb(x, "ncmf"))
ddi_df_subset.head()

In [ ]:
ddi_df_subset.rename(columns = {"drug_emb_ncmf": "drugA_emb_ncmf"}, inplace = True)
ddi_df_subset.head()

In [ ]:
ddi_df_subset["drugB_emb_ncmf"] = ddi_df_subset["drugB"].apply(lambda x: get_emb(x, "ncmf"))
ddi_df_subset.head()

In [ ]:
def get_concat_emb(row, alg):
    if alg == "ncmf":
        left = emb_dict[str(row["drugA"])]
        right = emb_dict[str(row["drugB"])]
    elif alg == "dcmf":
        left = emb_dict_dcmf[str(row["drugA"])]
        right = emb_dict_dcmf[str(row["drugB"])]
    emb_concat = np.concatenate((left, right))
    return list(emb_concat)

In [ ]:
ddi_df_subset["drug_pair_emb_ncmf"] = ddi_df_subset.apply(lambda row: get_concat_emb(row, "ncmf"), axis = 1)
ddi_df_subset.head()

In [ ]:
ddi_df_subset["drug_pair_emb_dcmf"] = ddi_df_subset.apply(lambda row: get_concat_emb(row, "dcmf"), axis = 1)
ddi_df_subset.head()

In [ ]:
ddi_df_subset[[f"emb_ncmf_{i}" for i in range(100)]] = pd.DataFrame(ddi_df_subset.drug_pair_emb_ncmf.tolist(), index=ddi_df_subset.index)
ddi_df_subset[[f"emb_dcmf_{i}" for i in range(200)]] = pd.DataFrame(ddi_df_subset.drug_pair_emb_dcmf.tolist(), index=ddi_df_subset.index)
ddi_df_subset.head()

In [ ]:
pca = PCA(n_components=5, random_state=42)
ddi_ncmf_pca = pca.fit_transform(ddi_df_subset[[f"emb_ncmf_{i}" for i in range(100)]].values)
ddi_dcmf_pca = pca.fit_transform(ddi_df_subset[[f"emb_dcmf_{i}" for i in range(200)]].values)
print(ddi_ncmf_pca.shape)
print(ddi_dcmf_pca.shape)

In [ ]:
ddi_df_subset["ncmf_clusters"] = hyp.cluster(ddi_ncmf_pca, n_clusters = k_poly)

In [ ]:
ddi_df_subset["dcmf_clusters"] = hyp.cluster(ddi_dcmf_pca, n_clusters = k_poly)

In [ ]:
ddi_df_subset.head()

#### Step 4 - FInd top k categories in each cluster

In [ ]:
# Single side effect dataset
topk_single_ncmf = {}
topk_single_dcmf = {}
for i in range(k_single):
    topk_single_ncmf[i] = single_se_df_subset[single_se_df_subset.ncmf_clusters == i]["Side Effect Name"].value_counts()[:topk_single].to_dict()
    topk_single_dcmf[i] = single_se_df_subset[single_se_df_subset.dcmf_clusters == i]["Side Effect Name"].value_counts()[:topk_single].to_dict()
print(topk_single_ncmf)
print(topk_single_dcmf)

In [ ]:
# Polypha side effect dataset
topk_poly_ncmf = {}
topk_poly_dcmf = {}
for i in range(k_poly):
    topk_poly_ncmf[i] = ddi_df_subset[ddi_df_subset.ncmf_clusters == i]["side_effect_category"].value_counts()[:topk_poly].to_dict()
    topk_poly_dcmf[i] = ddi_df_subset[ddi_df_subset.dcmf_clusters == i]["side_effect_category"].value_counts()[:topk_poly].to_dict()
print(topk_poly_ncmf)
print(topk_poly_dcmf)

#### Step 5 - Plot distribution of side effects in each cluster

In [ ]:
from textwrap import wrap

In [ ]:
# Single side effects
fig, ax = plt.subplots(k_single, 2, figsize=(25,25))
for i in range(k_single):
    sns.barplot(x = list(topk_single_ncmf[i].keys()), y = list(topk_single_ncmf[i].values()), ax = ax[i][0])
    ax[i][0].set_xticklabels(list(topk_single_ncmf[i].keys()), rotation=10)
    ax[i][0].set_title(f"NCMF cluster {i}")
    sns.barplot(x = list(topk_single_dcmf[i].keys()), y = list(topk_single_dcmf[i].values()), ax = ax[i][1])
    ax[i][1].set_xticklabels(list(topk_single_dcmf[i].keys()), rotation=10)
    ax[i][1].set_title(f"DCMF cluster {i}")
fig.suptitle(f"Single side effects - {k_single} most common in each cluster", y=1.005)
plt.tight_layout()

In [ ]:
# Polypharmacy side effects
fig, ax = plt.subplots(k_poly, 2, figsize=(25,25))
for i in range(k_poly):
    sns.barplot(x = list(topk_poly_ncmf[i].keys()), y = list(topk_poly_ncmf[i].values()), ax = ax[i][0])
    ax[i][0].set_xticklabels(list(topk_poly_ncmf[i].keys()), rotation=10)
    ax[i][0].set_title(f"NCMF cluster {i}")
    sns.barplot(x = list(topk_poly_dcmf[i].keys()), y = list(topk_poly_dcmf[i].values()), ax = ax[i][1])
    ax[i][1].set_xticklabels(list(topk_poly_dcmf[i].keys()), rotation=10)
    ax[i][1].set_title(f"DCMF cluster {i}")
fig.suptitle(f"Polypharmacy side effects - {k_poly} most common in each cluster", y=1.005)
plt.tight_layout()

#### Step 6 - Calculate ARI score and silhouette scores

In [ ]:
from sklearn.metrics import adjusted_rand_score, silhouette_score

In [ ]:
# Single side effect
print(f"NCMF adjusted rand score = {adjusted_rand_score(single_se_df_subset['Side Effect Name'], single_se_df_subset['ncmf_clusters'])}")
print(f"DCMF adjusted rand score = {adjusted_rand_score(single_se_df_subset['Side Effect Name'], single_se_df_subset['dcmf_clusters'])}")

In [ ]:
print(f"NCMF silhoeutte score for {k_single} clusters = {silhouette_score(single_se_df_subset[[f'emb_ncmf_{i}' for i in range(50)]], single_se_df_subset['ncmf_clusters'])}")
print(f"DCMF silhoeutte score for {k_single} clusters = {silhouette_score(single_se_df_subset[[f'emb_dcmf_{i}' for i in range(100)]], single_se_df_subset['dcmf_clusters'])}")

In [ ]:
# Polypharmacy side effect
print(f"NCMF adjusted rand score = {adjusted_rand_score(ddi_df_subset['side_effect_category'], ddi_df_subset['ncmf_clusters'])}")
print(f"DCMF adjusted rand score = {adjusted_rand_score(ddi_df_subset['side_effect_category'], ddi_df_subset['dcmf_clusters'])}")

In [ ]:
print(f"NCMF silhoeutte score for {k_poly} clusters = {silhouette_score(ddi_df_subset[[f'emb_ncmf_{i}' for i in range(50)]], ddi_df_subset['ncmf_clusters'])}")
print(f"DCMF silhoeutte score for {k_poly} clusters = {silhouette_score(ddi_df_subset[[f'emb_dcmf_{i}' for i in range(100)]], ddi_df_subset['dcmf_clusters'])}")

#### Step 7 - Visualize clusters using hypertools

In [ ]:
# Single side effect
fig, ax = plt.subplots(1, 2, figsize=(10,10))
fig.suptitle("Clusters for single side effect", y = 1.05)
hyp.plot(single_se_df_subset[[f"emb_ncmf_{i}" for i in range(50)]], ".", group=single_se_df_subset["ncmf_clusters"], ndims=2, title=f"NCMF {k_single} clusters", ax=ax[0])
hyp.plot(single_se_df_subset[[f"emb_dcmf_{i}" for i in range(100)]], ".", group=single_se_df_subset["dcmf_clusters"], ndims=2, title=f"DCMF {k_single} clusters", ax=ax[1])

In [ ]:
# Polypharmacy side effect
fig, ax = plt.subplots(1, 2, figsize=(10,10))
fig.suptitle("Clusters for polypharmacy side effect", y = 1.05)
hyp.plot(ddi_df_subset[[f"emb_ncmf_{i}" for i in range(100)]], ".", group=ddi_df_subset["ncmf_clusters"], ndims=2, title=f"NCMF {k_poly} clusters", ax=ax[0])
hyp.plot(ddi_df_subset[[f"emb_dcmf_{i}" for i in range(200)]], ".", group=ddi_df_subset["dcmf_clusters"], ndims=2, title=f"DCMF {k_poly} clusters", ax=ax[1])

#### Visualize existing labels using hypertools

In [ ]:
# Single side effect
fig, ax = plt.subplots(1, 2, figsize=(10,10))
fig.suptitle("Groups for single side effect", y = 1.05)
hyp.plot(single_se_df_subset[[f"emb_ncmf_{i}" for i in range(50)]], ".", group=single_se_df_subset["Side Effect Name"], ndims=2, title=f"NCMF side effect groups", ax=ax[0])
hyp.plot(single_se_df_subset[[f"emb_dcmf_{i}" for i in range(100)]], ".", group=single_se_df_subset["Side Effect Name"], ndims=2, title=f"DCMF side effect groups", ax=ax[1])

In [ ]:
# Polypharmacy side effect
fig, ax = plt.subplots(1, 2, figsize=(10,10))
fig.suptitle("Groups for polypharmacy side effect", y = 1.05)
hyp.plot(ddi_df_subset[[f"emb_ncmf_{i}" for i in range(100)]], ".", group=ddi_df_subset["side_effect_category"], ndims=2, title=f"NCMF side effect groups", ax=ax[0])
hyp.plot(ddi_df_subset[[f"emb_dcmf_{i}" for i in range(200)]], ".", group=ddi_df_subset["side_effect_category"], ndims=2, title=f"DCMF side effect groups", ax=ax[1])

#### Extra Visualization experiments

1. Read in drug-protein interactions from SNAP
2. Remove all duplicate drugs
3. Map all drugs in single side effect to the proteins they interact with
4. Cluster the drugs
5. For each cluster of drugs, find the protein that they interact with most

In [ ]:
# Load drug-protein associations from SNAP
drug_protein_df = pd.read_csv("./bio-decagon-targets.csv")
print(drug_protein_df.shape)
drug_protein_df.head()

In [ ]:
drug_protein_df_subset = drug_protein_df.drop_duplicates("STITCH")

In [ ]:
drug_protein_df_subset.Gene.value_counts()

In [ ]:
single_se_df_subset.head()

In [ ]:
single_se_df_subset["ncmf_pca1"] = single_se_ncmf_pca[:, 0]
single_se_df_subset["ncmf_pca2"] = single_se_ncmf_pca[:, 1]
single_se_df_subset["ncmf_pca3"] = single_se_ncmf_pca[:, 2]
single_se_df_subset["ncmf_pca4"] = single_se_ncmf_pca[:, 3]
single_se_df_subset["ncmf_pca5"] = single_se_ncmf_pca[:, 4]
single_se_df_subset["dcmf_pca1"] = single_se_dcmf_pca[:, 0]
single_se_df_subset["dcmf_pca2"] = single_se_dcmf_pca[:, 1]
single_se_df_subset["dcmf_pca3"] = single_se_dcmf_pca[:, 2]
single_se_df_subset["dcmf_pca4"] = single_se_dcmf_pca[:, 3]
single_se_df_subset["dcmf_pca5"] = single_se_dcmf_pca[:, 4]

single_se_df_subset["cmf_pca1"] = single_se_cmf_pca[:, 0]
single_se_df_subset["cmf_pca2"] = single_se_cmf_pca[:, 1]
single_se_df_subset["cmf_pca3"] = single_se_cmf_pca[:, 2]
single_se_df_subset["cmf_pca4"] = single_se_cmf_pca[:, 3]
single_se_df_subset["cmf_pca5"] = single_se_cmf_pca[:, 4]

single_se_df_subset["gcmf_pca1"] = single_se_gcmf_pca[:, 0]
single_se_df_subset["gcmf_pca2"] = single_se_gcmf_pca[:, 1]
single_se_df_subset["gcmf_pca3"] = single_se_gcmf_pca[:, 2]
single_se_df_subset["gcmf_pca4"] = single_se_gcmf_pca[:, 3]
single_se_df_subset["gcmf_pca5"] = single_se_gcmf_pca[:, 4]

single_se_df_subset["dfmf_pca1"] = single_se_dfmf_pca[:, 0]
single_se_df_subset["dfmf_pca2"] = single_se_dfmf_pca[:, 1]
single_se_df_subset["dfmf_pca3"] = single_se_dfmf_pca[:, 2]
single_se_df_subset["dfmf_pca4"] = single_se_dfmf_pca[:, 3]
single_se_df_subset["dfmf_pca5"] = single_se_dfmf_pca[:, 4]

In [ ]:
single_se_df_subset_with_proteins = single_se_df_subset.merge(drug_protein_df_subset, on = "STITCH")
single_se_df_subset_with_proteins.shape

In [ ]:
len(single_se_df_subset_with_proteins.Gene.unique())

In [ ]:
single_se_df_subset_with_proteins.head()

In [ ]:
single_se_df_subset_with_proteins["ncmf_clusters"] = hyp.cluster(single_se_df_subset_with_proteins[[f"ncmf_pca{i}" for i in range(1,6)]], n_clusters = k_single)
single_se_df_subset_with_proteins["dcmf_clusters"] = hyp.cluster(single_se_df_subset_with_proteins[[f"dcmf_pca{i}" for i in range(1,6)]], n_clusters = k_single)
single_se_df_subset_with_proteins["cmf_clusters"] = hyp.cluster(single_se_df_subset_with_proteins[[f"cmf_pca{i}" for i in range(1,6)]], n_clusters = k_single)
single_se_df_subset_with_proteins["gcmf_clusters"] = hyp.cluster(single_se_df_subset_with_proteins[[f"gcmf_pca{i}" for i in range(1,6)]], n_clusters = k_single)
single_se_df_subset_with_proteins["dfmf_clusters"] = hyp.cluster(single_se_df_subset_with_proteins[[f"dfmf_pca{i}" for i in range(1,6)]], n_clusters = k_single)

##### Finding top 5 proteins in each drug cluster

In [ ]:
# Single side effect dataset
topk_single_genes_ncmf = dict()
topk_single_genes_dcmf = dict()
topk_single_genes_cmf = dict()
topk_single_genes_gcmf = dict()
topk_single_genes_dfmf = dict()
for i in range(k_single):
    topk_single_genes_ncmf[i] = single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.ncmf_clusters == i]["Gene"].value_counts()[:topk_single].to_dict()
    topk_single_genes_dcmf[i] = single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.dcmf_clusters == i]["Gene"].value_counts()[:topk_single].to_dict()
    topk_single_genes_cmf[i] = single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.cmf_clusters == i]["Gene"].value_counts()[:topk_single].to_dict()
    topk_single_genes_gcmf[i] = single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.gcmf_clusters == i]["Gene"].value_counts()[:topk_single].to_dict()
    topk_single_genes_dfmf[i] = single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.dfmf_clusters == i]["Gene"].value_counts()[:topk_single].to_dict()
print(topk_single_genes_ncmf)
print(topk_single_genes_dcmf)
print(topk_single_genes_cmf)
print(topk_single_genes_gcmf)
print(topk_single_genes_dfmf)

In [ ]:
fig, ax = plt.subplots(k_single, 2, figsize=(10,10))
for i in range(k_single):
    sns.barplot(x = list(topk_single_genes_ncmf[i].keys()), y = list(topk_single_genes_ncmf[i].values()), ax=ax[i][0])
    ax[i][0].set_title(f"NCMF cluster {i}")
    ax[i][0].set_ylim(0, 65)
    x_list = list(sorted(topk_single_genes_ncmf[i].keys()))
    y_list = [topk_single_genes_ncmf[i][k] for k in x_list]
    for j in range(len(y_list)):
        ax[i][0].text(j, y_list[j]+2, str(y_list[j]), fontsize=14)
    ax[i][0].spines['top'].set_visible(False)
    ax[i][0].spines['right'].set_visible(False)
    sns.barplot(x = list(topk_single_genes_dcmf[i].keys()), y = list(topk_single_genes_dcmf[i].values()), ax=ax[i][1])
    ax[i][1].set_title(f"DCMF cluster {i}")
    ax[i][1].set_ylim(0, 65)
    x_list = list(sorted(topk_single_genes_dcmf[i].keys()))
    y_list = [topk_single_genes_dcmf[i][k] for k in x_list]
    for j in range(len(y_list)):
        ax[i][1].text(j, y_list[j]+2, str(y_list[j]), fontsize=14)
    ax[i][1].spines['top'].set_visible(False)
    ax[i][1].spines['right'].set_visible(False)
plt.tight_layout()
# plt.savefig("Polypharmacy_traditional_cluster_label_distribution.png")

#### Cluster visualization

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
# fig.suptitle("Clusters for drug embeddings", y = 1.1)

hyp.plot(single_se_df_subset_with_proteins[[f"emb_ncmf_{i}" for i in range(50)]], ".", group=single_se_df_subset_with_proteins["ncmf_clusters"], ndims=2, ax=ax[0])
ax[0].set_title("NCMF", fontsize=16)
ax[0].set_ylim(-0.9, 0.9)
ax[0].set_xlim(-0.99, 0.99)

hyp.plot(single_se_df_subset_with_proteins[[f"emb_dcmf_{i}" for i in range(100)]], ".", group=single_se_df_subset_with_proteins["dcmf_clusters"], ndims=2, ax=ax[1], legend = list(single_se_df_subset_with_proteins["dcmf_clusters"].unique()))
ax[1].set_title("DCMF", fontsize=16)
ax[1].set_ylim(-0.5, 0.9)
ax[1].set_xlim(-0.99, 0.999)

plt.legend(["cluster 0", "cluster 1", "cluster 2", "cluster 3", "cluster4"], bbox_to_anchor = (0, 1.2), loc="center", ncol=5, fontsize=16)
# plt.savefig("Polypharmacy_traditional_cluster_scatterplot.png", bbox_inches="tight")

##### Finding ARI score based on drug-protein association for each protein.

For each protein, create a list of size 645(for each drug), with value 0 if the protein is not associated with the drug from the drug-protein interaction matrix, and 1 otherwise. This is the true label. Also create a list that has value 1 if the protein is amongst the top 5 in the cluster the drug is associated with, 0 otherwise. This is the predicted label. These two lists are used to calculate ARI score for each protein.

In [ ]:
# get list of proteins
proteins_list_ncmf = []
proteins_list_dcmf = []
proteins_list_cmf = []
proteins_list_gcmf = []
proteins_list_dfmf = []
for i in range(k_single):
    for k in topk_single_genes_ncmf[i].keys():
        if k not in proteins_list_ncmf:
            proteins_list_ncmf.append(k)
    for j in topk_single_genes_dcmf[i].keys():
        if j not in proteins_list_dcmf:
            proteins_list_dcmf.append(j)
    for l in topk_single_genes_cmf[i].keys():
        if l not in proteins_list_cmf:
            proteins_list_cmf.append(l)
    for m in topk_single_genes_gcmf[i].keys():
        if m not in proteins_list_gcmf:
            proteins_list_gcmf.append(m)
    for n in topk_single_genes_dfmf[i].keys():
        if n not in proteins_list_dfmf:
            proteins_list_dfmf.append(n)

print(proteins_list_ncmf)
print(proteins_list_dcmf)
print(proteins_list_cmf)
print(proteins_list_gcmf)
print(proteins_list_dfmf)

In [ ]:
# creating true labels - we consider only the proteins in the top 5 of all clusters shown above.
true_labels_ncmf = [] # only for ncmf proteins
for p in proteins_list_ncmf:
    true_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    ones_idx = list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins["Gene"] == p].index)
    for o in ones_idx:
        true_label_p[o] = 1
    true_labels_ncmf.append(true_label_p)

In [ ]:
true_labels_ncmf = np.array(true_labels_ncmf)

In [ ]:
true_labels_dcmf = [] # only for dcmf proteins
for p in proteins_list_dcmf:
    true_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    ones_idx = list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins["Gene"] == p].index)
    for o in ones_idx:
        true_label_p[o] = 1
    true_labels_dcmf.append(true_label_p)

In [ ]:
true_labels_dcmf = np.array(true_labels_dcmf)

In [ ]:
true_labels_cmf = [] # only for cmf proteins
for p in proteins_list_cmf:
    true_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    ones_idx = list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins["Gene"] == p].index)
    for o in ones_idx:
        true_label_p[o] = 1
    true_labels_cmf.append(true_label_p)
true_labels_cmf = np.array(true_labels_cmf)

In [ ]:
true_labels_gcmf = [] # only for gcmf proteins
for p in proteins_list_gcmf:
    true_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    ones_idx = list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins["Gene"] == p].index)
    for o in ones_idx:
        true_label_p[o] = 1
    true_labels_gcmf.append(true_label_p)
true_labels_gcmf = np.array(true_labels_gcmf)

In [ ]:
true_labels_dfmf = [] # only for dfmf proteins
for p in proteins_list_dfmf:
    true_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    ones_idx = list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins["Gene"] == p].index)
    for o in ones_idx:
        true_label_p[o] = 1
    true_labels_dfmf.append(true_label_p)
true_labels_dfmf = np.array(true_labels_dfmf)

In [ ]:
# creating predicted labels from clusters
pred_labels_ncmf = []
for p in proteins_list_ncmf:
    pred_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    # all drugs that are part of the cluster where this protein belongs to gets a value 1
    clusters_p = []
    for k,v in topk_single_genes_ncmf.items():
        if p in v.keys():
            clusters_p.append(k)
    ones_idx = []
    for c in clusters_p:
        ones_idx.extend(list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.ncmf_clusters == c].index))
    for o in ones_idx:
        pred_label_p[o] = 1
    pred_labels_ncmf.append(pred_label_p)
    

In [ ]:
pred_labels_ncmf = np.array(pred_labels_ncmf)

In [ ]:
pred_labels_dcmf = []
for p in proteins_list_dcmf:
    pred_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    # all drugs that are part of the cluster where this protein belongs to gets a value 1
    clusters_p = []
    for k,v in topk_single_genes_dcmf.items():
        if p in v.keys():
            clusters_p.append(k)
    ones_idx = []
    for c in clusters_p:
        ones_idx.extend(list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.dcmf_clusters == c].index))
    for o in ones_idx:
        pred_label_p[o] = 1
    pred_labels_dcmf.append(pred_label_p)

In [ ]:
pred_labels_dcmf = np.array(pred_labels_dcmf)

In [ ]:
pred_labels_cmf = []
for p in proteins_list_cmf:
    pred_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    # all drugs that are part of the cluster where this protein belongs to gets a value 1
    clusters_p = []
    for k,v in topk_single_genes_cmf.items():
        if p in v.keys():
            clusters_p.append(k)
    ones_idx = []
    for c in clusters_p:
        ones_idx.extend(list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.cmf_clusters == c].index))
    for o in ones_idx:
        pred_label_p[o] = 1
    pred_labels_cmf.append(pred_label_p)
pred_labels_cmf = np.array(pred_labels_cmf)

In [ ]:
pred_labels_gcmf = []
for p in proteins_list_gcmf:
    pred_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    # all drugs that are part of the cluster where this protein belongs to gets a value 1
    clusters_p = []
    for k,v in topk_single_genes_gcmf.items():
        if p in v.keys():
            clusters_p.append(k)
    ones_idx = []
    for c in clusters_p:
        ones_idx.extend(list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.gcmf_clusters == c].index))
    for o in ones_idx:
        pred_label_p[o] = 1
    pred_labels_gcmf.append(pred_label_p)
pred_labels_gcmf = np.array(pred_labels_gcmf)

In [ ]:
pred_labels_dfmf = []
for p in proteins_list_dfmf:
    pred_label_p = [0] * single_se_df_subset_with_proteins.shape[0]
    # all drugs that are part of the cluster where this protein belongs to gets a value 1
    clusters_p = []
    for k,v in topk_single_genes_dfmf.items():
        if p in v.keys():
            clusters_p.append(k)
    ones_idx = []
    for c in clusters_p:
        ones_idx.extend(list(single_se_df_subset_with_proteins[single_se_df_subset_with_proteins.dfmf_clusters == c].index))
    for o in ones_idx:
        pred_label_p[o] = 1
    pred_labels_dfmf.append(pred_label_p)
pred_labels_dfmf = np.array(pred_labels_dfmf)

In [ ]:
ari_proteins_ncmf = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in NCMF clusters:")
for p in range(len(proteins_list_ncmf)):
    ari_proteins_ncmf = ari_proteins_ncmf.append({"protein": proteins_list_ncmf[p], "ari": adjusted_rand_score(true_labels_ncmf[p], pred_labels_ncmf[p])}, ignore_index=True)
ari_proteins_ncmf

In [ ]:
ari_proteins_ncmf.describe()["ari"]

In [ ]:
ari_proteins_dcmf = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in DCMF clusters:")
for p in range(len(proteins_list_dcmf)):
    ari_proteins_dcmf = ari_proteins_dcmf.append({"protein": proteins_list_dcmf[p], "ari": adjusted_rand_score(true_labels_dcmf[p], pred_labels_dcmf[p])}, ignore_index=True)
ari_proteins_dcmf

In [ ]:
ari_proteins_dcmf.describe()["ari"]

In [ ]:
ari_proteins_cmf = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in CMF clusters:")
for p in range(len(proteins_list_cmf)):
    ari_proteins_cmf = ari_proteins_cmf.append({"protein": proteins_list_cmf[p], "ari": adjusted_rand_score(true_labels_cmf[p], pred_labels_cmf[p])}, ignore_index=True)
ari_proteins_cmf.describe()["ari"]

In [ ]:
ari_proteins_gcmf = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in gCMF clusters:")
for p in range(len(proteins_list_gcmf)):
    ari_proteins_gcmf = ari_proteins_gcmf.append({"protein": proteins_list_gcmf[p], "ari": adjusted_rand_score(true_labels_gcmf[p], pred_labels_gcmf[p])}, ignore_index=True)
ari_proteins_gcmf.describe()["ari"]

In [ ]:
ari_proteins_dfmf = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in DFMF clusters:")
for p in range(len(proteins_list_dfmf)):
    ari_proteins_dfmf = ari_proteins_dfmf.append({"protein": proteins_list_dfmf[p], "ari": adjusted_rand_score(true_labels_dfmf[p], pred_labels_dfmf[p])}, ignore_index=True)
ari_proteins_dfmf.describe()["ari"]

In [ ]:
# silhoeutte score
print(f"NCMF silhoeutte score {silhouette_score(single_se_df_subset_with_proteins[[f'ncmf_pca{i}' for i in range(1,6)]], single_se_df_subset_with_proteins['ncmf_clusters'])}")
print(f"DCMF silhoeutte score {silhouette_score(single_se_df_subset_with_proteins[[f'dcmf_pca{i}' for i in range(1,6)]], single_se_df_subset_with_proteins['dcmf_clusters'])}")

In [ ]:
ari_proteins_ncmf.merge(ari_proteins_dcmf, on="protein", suffixes=("_ncmf", "_dcmf")).merge(ari_proteins_cmf, on="protein").rename(columns={"ari": "ari_cmf"}).merge(ari_proteins_gcmf, on="protein").rename(columns={"ari": "ari_gcmf"}).merge(ari_proteins_dfmf, on="protein").rename(columns={"ari": "ari_dfmf"})

In [ ]:
ari_proteins_ncmf.merge(ari_proteins_dcmf, on="protein", suffixes=("_ncmf", "_dcmf")).merge(ari_proteins_cmf, on="protein").rename(columns={"ari": "ari_cmf"}).merge(ari_proteins_gcmf, on="protein").rename(columns={"ari": "ari_gcmf"}).merge(ari_proteins_dfmf, on="protein").rename(columns={"ari": "ari_dfmf"}).describe()

In [ ]:
# method 2 pred labels are the same as the cluster labels
pred_labels_ncmf_2 = np.array(single_se_df_subset_with_proteins["ncmf_clusters"])
pred_labels_dcmf_2 = np.array(single_se_df_subset_with_proteins["dcmf_clusters"])
pred_labels_cmf_2 = np.array(single_se_df_subset_with_proteins["cmf_clusters"])
pred_labels_gcmf_2 = np.array(single_se_df_subset_with_proteins["gcmf_clusters"])
pred_labels_dfmf_2 = np.array(single_se_df_subset_with_proteins["dfmf_clusters"])

In [ ]:
ari_proteins_ncmf_2 = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in NCMF clusters:")
for p in range(len(proteins_list_ncmf)):
    ari_proteins_ncmf_2 = ari_proteins_ncmf_2.append({"protein": proteins_list_ncmf[p], "ari": adjusted_rand_score(true_labels_ncmf[p], pred_labels_ncmf_2)}, ignore_index=True)
ari_proteins_ncmf_2

In [ ]:
ari_proteins_dcmf_2 = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in DCMF clusters:")
for p in range(len(proteins_list_dcmf)):
    ari_proteins_dcmf_2 = ari_proteins_dcmf_2.append({"protein": proteins_list_dcmf[p], "ari": adjusted_rand_score(true_labels_dcmf[p], pred_labels_dcmf_2)}, ignore_index=True)
ari_proteins_dcmf_2

In [ ]:
ari_proteins_ncmf_2.describe()["ari"]

In [ ]:
ari_proteins_dcmf_2.describe()["ari"]

In [ ]:
merged = ari_proteins_ncmf_2.merge(ari_proteins_dcmf_2, on="protein", suffixes=("_ncmf", "_dcmf"))

In [ ]:
ari_proteins_cmf_2 = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in CMF clusters:")
for p in range(len(proteins_list_cmf)):
    ari_proteins_cmf_2 = ari_proteins_cmf_2.append({"protein": proteins_list_cmf[p], "ari": adjusted_rand_score(true_labels_cmf[p], pred_labels_cmf_2)}, ignore_index=True)
ari_proteins_cmf_2

In [ ]:
ari_proteins_gcmf_2 = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in gCMF clusters:")
for p in range(len(proteins_list_gcmf)):
    ari_proteins_gcmf_2 = ari_proteins_gcmf_2.append({"protein": proteins_list_gcmf[p], "ari": adjusted_rand_score(true_labels_gcmf[p], pred_labels_gcmf_2)}, ignore_index=True)
ari_proteins_gcmf_2

In [ ]:
ari_proteins_dfmf_2 = pd.DataFrame(columns = ["protein", "ari"])
print("ARI scores for proteins in DFMF clusters:")
for p in range(len(proteins_list_dfmf)):
    ari_proteins_dfmf_2 = ari_proteins_dfmf_2.append({"protein": proteins_list_dfmf[p], "ari": adjusted_rand_score(true_labels_dfmf[p], pred_labels_dfmf_2)}, ignore_index=True)
ari_proteins_dfmf_2

In [ ]:
merged = merged.merge(ari_proteins_cmf_2, on="protein").rename(columns={"ari": "ari_cmf"}).merge(ari_proteins_gcmf_2, on="protein").rename(columns={"ari": "ari_gcmf"}).merge(ari_proteins_dfmf_2, on="protein").rename(columns={"ari": "ari_dfmf"})

In [ ]:
merged.describe()